In [ ]:
import os
import re
import json
import warnings
from pathlib import Path
from typing import List, Dict, Any

try:
    from dotenv import load_dotenv; load_dotenv()
except ImportError:
    pass

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

warnings.filterwarnings("ignore")
os.makedirs("data", exist_ok=True)
os.makedirs("db/cafe_db", exist_ok=True)

try:
    from langchain_core.tools import tool
    from langchain_core.runnables import chain
    from langchain_core.documents import Document
    from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings
    from langchain_community.vectorstores import FAISS
    from langchain_community.tools import TavilySearchResults
except Exception:
    tool, chain, Document, ChatOpenAI, OpenAIEmbeddings, FAISS, TavilySearchResults = [None] * 7

try:
    import wikipedia
    wikipedia.set_lang("ko")
except Exception:
    wikipedia = None

BASE_DIR = Path(os.getcwd())
DATA_DIR = BASE_DIR / "data"
DB_DIR = BASE_DIR / "db" / "cafe_db"
MENU_FILE = DATA_DIR / "cafe_menu_data.txt" 

def _read_menu_raw_content() -> str:
    """메뉴 파일을 읽고 단일 raw string으로 반환 (경로 탐색 강화)"""
    candidates = [
        MENU_FILE, 
        Path.cwd() / "data" / "cafe_menu_data.txt",
        Path(__file__).resolve().parent / "data" / "cafe_menu_data.txt" if '__file__' in locals() or '__file__' in globals() else None
    ]
    for p in [c for c in candidates if c]:
        if p.exists():
            return p.read_text(encoding="utf-8")
            
    raise FileNotFoundError(
        "메뉴 파일 누락: './data/cafe_menu_data.txt' 파일을 찾을 수 없습니다. 경로를 확인하세요."
    )

def _split_menu_by_number(raw_text: str) -> List[str]:
    """
    강화된 정규식을 사용하여 메뉴 번호(1., 2., ...)를 기준으로 항목을 분리합니다.
    """
    normalized_text = raw_text.replace('\r\n', '\n').replace('\r', '\n')
    pattern = r'(\d+\.\s*.*?)((?=\n\d+\.)|\Z)'
    
    menu_items_tuples = re.findall(pattern, normalized_text, re.DOTALL)
    menu_items = [item[0].strip() for item in menu_items_tuples if item[0].strip()]

    if not menu_items or len(menu_items) < 3:
        raise ValueError(f"메뉴 분리에 실패했습니다. 분리된 항목 수: {len(menu_items)}. 파일 형식을 확인하세요.")
        
    return menu_items

def _ensure_vector_db() -> "FAISS":
    """FAISS DB를 생성하고 저장합니다. (save_local 인자 수정)"""
    if not all([FAISS, OpenAIEmbeddings, Document]):
        raise RuntimeError("DB 의존성(FAISS/OpenAIEmbeddings)이 부족합니다.")
    
    raw_content = _read_menu_raw_content()
    lines = _split_menu_by_number(raw_content) 
    
    DB_DIR.mkdir(parents=True, exist_ok=True)

    docs = []
    for i, line in enumerate(lines, start=1):
        parts = line.split('\n', 1)
        menu_name = re.sub(r'^\d+\.\s*', '', parts[0].strip()).strip()
        docs.append(Document(page_content=line, metadata={"name": menu_name or f"item_{i}"}))

    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vectordb = FAISS.from_documents(docs, embeddings)
    
    vectordb.save_local(str(DB_DIR)) 
    print(f"DB 생성 완료: 총 {len(docs)}개 항목")
    return vectordb

def _load_vector_db() -> "FAISS":
    """FAISS DB를 로드하며, 없으면 새로 생성합니다. (load_local 인자 유지)"""
    if not all([FAISS, OpenAIEmbeddings]):
        raise RuntimeError("DB 로드 의존성(FAISS/OpenAIEmbeddings)이 부족합니다.")
        
    if not (DB_DIR / "index.faiss").exists():
        return _ensure_vector_db()
        
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
   
    return FAISS.load_local(str(DB_DIR), embeddings, allow_dangerous_deserialization=True) 

_TAVILY_TOOL_OBJ = TavilySearchResults(max_results=3, search_depth="advanced") if TavilySearchResults else None

if tool is not None and wikipedia is not None:
    @tool
    def wiki_summary(topic: str) -> str:
        """위키피디아에서 일반 지식(예: 커피 역사, 제조 방법)을 검색하고 요약해야 할 때 사용하세요."""
        try:
            return wikipedia.summary(topic, sentences=3, auto_suggest=False, redirect=True)
        except Exception as e:
            return f"Wikipedia 검색 오류: {e}"
else:
    def wiki_summary(topic: str) -> str:
        return "Wikipedia 툴 비활성화."

if tool is not None:
    @tool
    def db_search_cafe_func(query: str) -> List["Document"]:
        """로컬 카페 메뉴 DB에서 특정 메뉴의 가격, 재료, 설명과 같은 정보를 검색해야 할 때 사용하세요."""
        try:
            vectordb = _load_vector_db()
            retriever = vectordb.as_retriever(search_kwargs={"k": 3})
            return retriever.invoke(query)
        except Exception as e:
            print(f"경고: DB 검색 실패 ({e}). 파일 내용으로 폴백합니다.")
            try:
                lines = _split_menu_by_number(_read_menu_raw_content())
                q_compact = query.replace(" ", "")
                matched = [ln for ln in lines if q_compact in ln.replace(" ", "")]
                return [Document(page_content=ln, metadata={}) for ln in matched]
            except Exception:
                return []
else:
    def db_search_cafe_func(query: str) -> List[str]:  # type: ignore
        return []

def _make_llm_with_tools():
    """도구가 바인딩된 LLM 생성."""
    if not all([ChatOpenAI, OPENAI_API_KEY]): return None, []
        
    llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
    tools = [t for t in [_TAVILY_TOOL_OBJ, wiki_summary, db_search_cafe_func] if t is not None]
    
    try:
        llm_with_tools = llm.bind_tools(tools)
    except Exception:
        llm_with_tools = None
        
    return llm_with_tools, tools

def _make_llm_text_only():
    """최종 답변을 위한 툴이 없는 순수 텍스트 LLM 생성."""
    if not all([ChatOpenAI, OPENAI_API_KEY]): return None
    return ChatOpenAI(model=OPENAI_MODEL, temperature=0)

def _format_docs(docs: List[Any]) -> str:
    """Document 객체를 읽기 쉬운 텍스트로 변환"""
    chunks = [getattr(d, "page_content", str(d)) for d in docs]
    return "\n===\n".join(chunks)

def _compose_answer_from_db(db_output: str, question: str) -> str:
    """DB 결과에서 가격, 재료, 특징을 추출하여 답변 (LLM 실패 시 폴백)"""
    first_item = db_output.split("\n===\n")[0].strip()
    if not first_item: return "메뉴 DB에서 관련 정보를 찾지 못했습니다."
        
    menu_match = re.search(r'^\d+\.\s*(.*?)\n', first_item, re.MULTILINE)
    price_match = re.search(r'•\s*가격:\s*(₩[\d,]+)', first_item)
    ingredients_match = re.search(r'•\s*주요 원료:\s*(.*?)\n', first_item)
    desc_match = re.search(r'•\s*설명:\s*(.*?)(?=\n\d+\.|\Z)', first_item, re.DOTALL)
    
    menu_name = menu_match.group(1).strip() if menu_match else "요청 메뉴"
    parts = []
    if price_match: parts.append(f"가격: {price_match.group(1)}")
    if ingredients_match: parts.append(f"주요 재료: {ingredients_match.group(1).strip()}")
    if desc_match: 
        desc_text = desc_match.group(1).strip().replace('\n', ' ')
        parts.append(f"특징: {desc_text}")
    
    return f"'{menu_name}'의 정보입니다.\n" + " / ".join(parts)

def _safe_tool_invoke(tool_obj: Any, name: str, args: Any, question: str) -> Dict[str, Any]:
    """개별 툴을 안전하게 실행하고 결과를 딕셔너리로 반환"""
    try:
        q = (args or {}).get("query", question) if isinstance(args, dict) else question
        
        if name == "db_search_cafe_func":
            out_docs = tool_obj.invoke(q)
            out = _format_docs(out_docs)
        elif name == "wiki_summary":
            query_arg = (args or {}).get("topic", q) if isinstance(args, dict) else q
            out = tool_obj.invoke(query_arg)
        elif name in ["tavily_search_results_json", "TavilySearchResults"]:
            out = tool_obj.invoke({"query": q})
        else:
            out = tool_obj.invoke(q)
             
        return {"tool": name, "output": out}
        
    except Exception as e:
        return {"tool": name, "output": f"ERROR: 툴 실행 실패 - {e}"}

if chain is not None and ChatOpenAI is not None:
    @chain
    def tool_calling_chain(inputs: Dict[str, Any]) -> Dict[str, Any]:
        question = inputs["question"]
        
        if OPENAI_API_KEY is None:
            return {"answer": "OpenAI API 키가 설정되지 않아 답변을 생성할 수 없습니다.", "tool_outputs": []}

        llm_with_tools, tools = _make_llm_with_tools()
        registry: Dict[str, Any] = {
            getattr(t, "name", getattr(t, "__name__", None)): t for t in tools if getattr(t, "name", getattr(t, "__name__", None))
        }

        try:
            ai_msg = llm_with_tools.invoke([HumanMessage(content=question)])
        except Exception as e:
            return {"answer": f"LLM 호출 오류 발생: {e}", "tool_outputs": []}
            
        tool_outputs = []
        calls = getattr(ai_msg, "tool_calls", []) or []

        for call in calls:
            name = getattr(call, "name", None) or (call.get("name") if isinstance(call, dict) else None)
            args = getattr(call, "args", {})
            tool_obj = registry.get(name)
            
            if tool_obj:
                tool_outputs.append(_safe_tool_invoke(tool_obj, name, args, question))

        has_db = any(o.get("tool") == "db_search_cafe_func" for o in tool_outputs)
        if ("가격" in question or "메뉴" in question or "특징" in question or "재료" in question) and not has_db:
            db_obj = registry.get("db_search_cafe_func")
            if db_obj:
                print("정보 보강: DB 검색 결과 강제 추가")
                tool_outputs.append(_safe_tool_invoke(db_obj, "db_search_cafe_func", {}, question))
        
        llm_text = _make_llm_text_only()
        evidence = json.dumps(tool_outputs, ensure_ascii=False, indent=2)
        
        final_prompt = (
            "당신은 카페 메뉴 안내 AI입니다. 다음 도구 실행 결과를 바탕으로 질문에 한국어로 정확히 답하세요.\n"
            "가격(₩ 표기), 재료, 특징을 포함해 간결하게 정리하세요. 추측하지 말고 도구 결과만 사용하세요.\n"
            "답변은 텍스트로만 출력하세요. 도구를 호출하지 마세요.\n"
            f"질문: {question}\n"
            f"도구결과(JSON):\n{evidence}\n"
        )
        
        try:
            final_msg = llm_text.invoke(final_prompt)
            content = getattr(final_msg, "content", None)
            
            if not content:
                content = _compose_answer_from_db(evidence, question)
                
            return {"answer": content, "tool_outputs": tool_outputs}
            
        except Exception:

            content = _compose_answer_from_db(evidence, question)
            return {"answer": content + " (LLM 최종 생성 실패로 DB 답변으로 대체됨)", "tool_outputs": tool_outputs}

else:

    def tool_calling_chain(inputs: Dict[str, Any]) -> Dict[str, Any]:
        return {"answer": "핵심 종속성이 부족하여 체인을 실행할 수 없습니다.", "tool_outputs": []}


def run_test():
    print("===== 문제 4-1 Tool Calling 체인 테스트 시작 =====")
    
    question_db = "카푸치노의 가격과 특징은 무엇인가요?"
    print(f"\n[테스트 1] 질문: {question_db}")
    result_db = tool_calling_chain.invoke({"question": question_db})
    print("\n== 최종 답변 ==")
    print(result_db.get("answer", ""))
    
    question_wiki = "카푸치노의 유래에 대해서 알려주세요."
    print(f"\n[테스트 2] 질문: {question_wiki}")
    result_wiki = tool_calling_chain.invoke({"question": question_wiki})
    print("\n== 최종 답변 ==")
    print(result_wiki.get("answer", ""))

    question_web = "2024년 최신 커피 트렌드는 무엇인가요?"
    print(f"\n[테스트 3] 질문: {question_web}")
    result_web = tool_calling_chain.invoke({"question": question_web})
    print("\n== 최종 답변 ==")
    print(result_web.get("answer", ""))


if __name__ == "__main__":
    try:
        _load_vector_db() 
    except FileNotFoundError as e:
        print("\n" + "=" * 50)
        print("FATAL: 메뉴 파일을 찾을 수 없습니다.")
        print(f"오류: {e}")
        print("파일명을 'cafe_menu_data.txt'로 변경하고 './data' 폴더에 저장했는지 확인하세요.")
        print("=" * 50)
        exit(1)
    except Exception as e:
        print("\n" + "=" * 50)
        print("FATAL: DB 구축 실패 (데이터 형식 또는 기타 오류)")
        print(f"오류: {e}")
        print("\n'cafe_menu_data.txt' 파일 내용이 메뉴 번호(1., 2., 3. ...)로 시작하는지, 그리고 패키지(FAISS 등)가 정상 설치되었는지 확인하세요.")
        print("=" * 50)
        exit(1)
    
    if ChatOpenAI is not None and OPENAI_API_KEY is not None:
        run_test()
    else:
        print("\n" + "=" * 50)
        print(" OpenAI API 키 누락 또는 LangChain/OpenAI 패키지 미설치")
        print("API 키를 환경 변수에 설정하거나, 필요한 패키지를 설치하세요.")
        print("=" * 50)

===== 문제 4-1 Tool Calling 체인 테스트 시작 =====

[테스트 1] 질문: 카푸치노의 가격과 특징은 무엇인가요?

== 최종 답변 ==
카푸치노의 가격은 ₩5,000입니다. 주요 원료는 에스프레소, 스팀 밀크, 우유 거품이며, 이탈리아 전통 커피로 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성되어 있습니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 특징이며, 계피 파우더를 뿌려 제공합니다.

[테스트 2] 질문: 카푸치노의 유래에 대해서 알려주세요.

== 최종 답변 ==
카푸치노의 유래에 대한 정보는 제공할 수 없습니다. 다른 질문이 있으시면 말씀해 주세요.

[테스트 3] 질문: 2024년 최신 커피 트렌드는 무엇인가요?

== 최종 답변 ==
2024년 커피 트렌드는 '투게더(TOGETHER)'로 선정되었습니다. 이 키워드는 커피의 생산과 유통 과정에서 다양한 요소가 서로 화합하고 상생하는 모습을 강조합니다. 서울카페쇼에서는 지속 가능한 카페 산업의 성장을 위한 방향성을 제시하며, 맞춤형 경험을 제공하는 브랜드가 증가하고 있습니다. 소비자들은 개인의 취향을 나노 단위로 분석하고, 다국적 메뉴와 문화 이벤트를 즐기는 경향이 높아지고 있습니다. 또한, 반려동물 친화적인 카페가 늘어나고 있으며, 대표적인 프랜차이즈 카페들이 새로운 라이프스타일을 반영한 매장을 확대하고 있습니다.
